In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 19.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=bacb46fc69af5e039d06b21c25bed37513177dfad000f2886a337fe3986ad5b3
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession as ss
spark = ss.builder.getOrCreate()

In [ ]:
# os.chdir('Annotated')
files = [file for file in os.listdir() if not file.startswith('.')]
files.remove('sentiment140')

In [ ]:
files

['total_combined_text_and_ids_preprocessed_upd_res',
 'SMILE_Twitter_Emotion_dataset_cleaned_preprocessed',
 'Brands_and_Product_Emotions_cleaned.csv_preprocessed',
 'The_Valence_and_Arousal_Facebook_Posts_cleaned_preprocessed',
 'Affect_data_cleaned.csv_preprocessed',
 'Sentiment_Labelled_Sentences_Data_Set_cleaned_preprocessed',
 'Sentiment_Analysis_in_Text_cleaned.csv_preprocessed',
 'Sentiment_Emotion_Mining_Toolkit_(EMTk)_cleaned_preprocessed',
 'Primary_Emotions_of_Statements_cleaned_preprocessed',
 'WASSA_2017_Shared_Task_on_Emotion_Intensity_cleaned.csv_preprocessed',
 'IMDB_cleaned.csv_preprocessed',
 'sentiment140-combined']

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Annotated/EmoBank_cleaned.csv_preprocessed",header = True, multiLine=True,inferSchema=True)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,FloatType
class cal_mean_sentiment:
  def __init__(self):
    # self.path_read = '/content/drive/MyDrive/Annotated/'
    self.path = '/content/drive/MyDrive/Annotated/mean_sent_lex/'
    
  @staticmethod
  @udf(returnType = StringType())
  def classify(score):
      if score>0.5:
          return 'VPos'
      if score>0 and score<=0.5:
          return 'Pos'
      if score<0 and score>=-0.5:
          return 'Neg'
      if score<-0.5:
          return 'VNeg'
      return 'Neu'

  def process(self,f):
    self.df = spark.read.csv(self.path+f,header = True,multiLine=True,inferSchema=True)
    self.df = self.df.dropna()
    # self.df = self.df.withColumn("mean_sent_score", (col("vader_score")+col("blob_score")+col("afinn_score"))/3)
    # unlist = udf(lambda x: float(list(x)[0]), FloatType())
    # self.df = self.df.withColumn('mean_sent_score_f',unlist('mean_sent_score'))
    # self.df = self.df.withColumnRenamed('mean_sent_score_f','mean_sent_score')
    self.df = self.df.withColumn("final_sent_class", cal_mean_sentiment.classify("mean_sent_score"))
    print("SAVING -- "+f)
    self.df.write.mode("overwrite").option("header","true").csv(self.path+'final_sent_class/'+f)
    print("SAVED -- "+f)



In [ ]:
c = cal_mean_sentiment()
c.process(files[11])
# path = '/content/drive/MyDrive/Annotated/mean_sent_lex/'
# df = spark.read.csv(path+files[11],header = True, multiLine=True,inferSchema=True)


SAVING -- sentiment140-combined
SAVED -- sentiment140-combined


In [ ]:
df.dropna().count()

1568971

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

# unionAll(df_summerfruits, df_fruits, df_dryfruits).distinct().show()

In [ ]:
df0 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140-combined",header = True, multiLine=True,inferSchema=True)
# df1 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140/part-1",header = True, escape="\"",multiLine=True,inferSchema=True)
# df2 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140/part-2",header = True, escape="\"",multiLine=True,inferSchema=True)
# df3 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140/part-3",header = True, escape="\"",multiLine=True,inferSchema=True)
# df4 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140/part-4",header = True, escape="\"",multiLine=True,inferSchema=True)
# df5 = spark.read.csv("/content/drive/MyDrive/Annotated/sentiment140/part-5",header = True, escape="\"",multiLine=True,inferSchema=True)

In [ ]:
comb_df = unionAll(df0,df1,df2,df3,df4,df5).distinct()

In [ ]:
comb_df = comb_df.drop_duplicates().dropna()

In [ ]:
print(df0.count())
# print(df1.count())
# print(df2.count())
# print(df3.count())
# print(df4.count())
# print(df5.count())
# print(comb_df.count())

1569317


In [ ]:
comb_df.write.mode("overwrite").option("header","true").csv('/content/drive/MyDrive/Annotated/sentiment140-combined')